In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.utils import to_categorical
from keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Flatten
from keras.models import Model
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from keras import backend as K
from skimage.transform import resize
from tqdm import tqdm
import cv2
import gc

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precision_val = precision(y_true, y_pred)
    recall_val = recall(y_true, y_pred)
    return 2*((precision_val*recall_val)/(precision_val+recall_val+K.epsilon()))

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Resize images to meet InceptionV3 input size requirements (75x75)
train_resized_imgs = []
for img in tqdm(x_train):
   resized = cv2.resize(img,(75,75))
   train_resized_imgs.append(resized)
train_resized_imgs = np.array(train_resized_imgs)
x_train = train_resized_imgs


test_resized_imgs = []
for img in tqdm(x_test):
   resized = cv2.resize(img,(75,75))
   test_resized_imgs.append(resized)
test_resized_imgs = np.array(test_resized_imgs)
x_test = test_resized_imgs

del train_resized_imgs
del test_resized_imgs
gc.collect()

# Preprocess the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

100%|██████████| 10000/10000 [00:00<00:00, 31399.24it/s]


In [ ]:
# Parameters
epochs = 30
batch_size = 128
learning_rate = 0.001
momentum = 0.9

# Load pre-trained InceptionV3 model
base_model = InceptionV3(weights= "imagenet", include_top=False, input_shape=(75, 75, 3))

# Freeze layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of pre-trained model
x = base_model.output
# x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
# x = Flatten()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(100, activation='softmax')(x)

# Combine base model and custom layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)


# Compile the model with custom metrics
model.compile(optimizer=SGD(learning_rate=learning_rate, momentum= momentum),
              loss='categorical_crossentropy',
              metrics=['accuracy', recall, precision, f1])

# Train the model
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=1)

# Plot training and validation accuracy
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.show()

NameError: name 'InceptionV3' is not defined

# **Epochs: 30     Learning Rate: 0.01     Momentum: 0.9    Batch size: 128**